# Anonymous user login using ecdsa

In [48]:
import sha3
import json

from collections import namedtuple
from ecdsa import SigningKey, VerifyingKey, NIST256p
from IPython.display import display, HTML

In [49]:
# get parameters from user
passphrase = 'Connors test.'
cycles = 7


In [50]:
# Client side generation of keys
def sha(str_in):
    keccak = sha3.keccak_256()
    keccak.update(str_in)
    return keccak.hexdigest()[:32]

priv_key_string = sha(passphrase)
for c in range(cycles):
    priv_key_string = sha(priv_key_string)
priv_key = SigningKey.from_string(priv_key_string, curve=NIST256p)
pub_key = priv_key.get_verifying_key()

private_key = priv_key.to_string().encode('hex')
public_key = pub_key.to_string().encode('hex')
address = sha(pub_key.to_string().encode('hex'))


print "Private key: 0x" + private_key
print "Public key:  0x" + public_key[:64] + '... (shortened for brevity)'
print "Address:     0x" + address

# Store private_key to localStorage or local file (never let server see)
LocalStorage = namedtuple('local_storage', ('address', 'private_key'))
local_storage = LocalStorage(address, private_key)

Private key: 0x3932363832636530396136383936373234616264373262303264643431623062
Public key:  0xb8b495406018b5e6f17893ee2e5b704791ab2eb9d68d34ae6b5c0132d6ab5bca... (shortened for brevity)
Address:     0x272ca12fa07469a00f7f54770459b02e


In [51]:
# Example server
from flask import Flask, request, jsonify

app = Flask('anonymous_login_api')
client = app.test_client()
users_registry = {}

@app.route('/<address>', methods=('GET',))
def lookup_user(address):
    return jsonify(users_registry.get(address, None))

In [53]:
# Because registry was empty we should get no user for our addres
res = client.get('/' + local_storage.address)
data = json.loads(res.data)
data is None

True